In [23]:
import numpy as np
import pandas as pd
import sys, os, copy, cv2
sys.path.append('/home/rahim/Desktop/projects/mrob/build/mrobpy/')
import mrob
import matplotlib.pyplot as plt
import open3d as o3d
from tqdm import tqdm

In [24]:
fx=1.72435023e03
fy=1.72690135e03
cx=4.92790143e02
cy=9.59556831e02

camera_matrix_s20 = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
dist_coeff_s20 = np.zeros(5)

In [25]:
mobile_pose_dir = '/home/rahim/patata/datasets/navigine/mocab/apriltag/mobile/new/15November/mobile_color_gt.csv'

In [26]:
def quat_to_SE3(quat_pose):
    
    rot_4x1 = quat_pose[-4:]
    tra_3x1 = quat_pose[1:4]

    rot = mrob.geometry.quat_to_so3(rot_4x1)
    pose = mrob.geometry.SE3(mrob.geometry.SO3(rot),tra_3x1)

    return pose

In [27]:
def ax(pose, size = 0.1):
    axis = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    return axis.transform(pose)

In [28]:
mocap_poses = pd.read_csv(mobile_pose_dir, header=None, sep="\s+")
mocap_quat = np.array([list(map(float, np.asarray(each)[0].split(",")[:-2])) for _,each in mocap_poses.iterrows()])

In [29]:
mocap = np.zeros((mocap_quat.shape[0],4,4))
# orb = np.zeros((mocap_quat.shape[0],4,4))
for i in range(len(mocap_quat)):
    mocap[i,:,:] = (quat_to_SE3(mocap_quat[i,:])).T()
    # orb[i,:,:] = (quat_to_SE3(orb_quat[i,:])).T()



In [46]:
# !pip install opencv-contrib-python

In [72]:

def main(color_dir, camera_matrix, dist_coeff, idx):

    aruco_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_100)
    # Note: Pattern generated using the following link
    # https://calib.io/pages/camera-calibration-pattern-generator
    board = cv2.aruco.CharucoBoard_create(14, 9, 0.03975, 0.02975, aruco_dict)
    # (columns:squaresX, rows:squaresY, Checker Width: squareLength, markerLength, markers type)

    all_corners, all_ids, T_arucos = [], [], []
    frame = cv2.imread(color_dir)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, rejectedImgPoints = cv2.aruco.detectMarkers(gray, aruco_dict)
    if len(corners) > 0:
        ret, c_corners, c_ids = cv2.aruco.interpolateCornersCharuco(corners, ids, gray, board)
        if ret > 0:
            all_corners.append(c_corners)
            all_ids.append(c_ids)
    imsize = gray.shape

    all_corners = [x for x in all_corners if len(x) >= 4]
    all_ids = [x for x in all_ids if len(x) >= 4]
    ret, _, _, rvec, tvec = cv2.aruco.calibrateCameraCharuco(
        all_corners, all_ids, board, imsize, None, None)

    ret, p_rvec, p_tvec = cv2.aruco.estimatePoseCharucoBoard(charucoCorners=c_corners,
                                                                charucoIds=c_ids,
                                                                board=board,
                                                                cameraMatrix=camera_matrix,
                                                                distCoeffs=dist_coeff, 
                                                                rvec=0*rvec[0], 
                                                                tvec=0*tvec[0],
                                                                )
    if ret>0:                                                           
        ax = cv2.drawFrameAxes(frame,
                                camera_matrix,
                                dist_coeff,
                                p_rvec,
                                p_tvec,
                                0.1)
        cv2.imwrite(f'/home/rahim/Desktop/projects/mrob/notebooks/figs/{idx}.png', np.asarray(ax))

    T_aruco = mrob.geometry.SE3(mrob.geometry.SO3(p_rvec),p_tvec)
    return T_aruco, ret, ax



$$^{m}T_{af}^{-1} \cdot ^{m}T_f = ^{af}T_a \cdot ^{c}T_a^{-1} \cdot ^{c}T_f$$

$$^{af}T_f = Y \cdot ^{a}T_c \cdot X$$

In [94]:
import cv2
AT_C = []
MT_F = []
imgs = []
import glob
colors = sorted(glob.glob('/home/rahim/patata/datasets/navigine/mocab/apriltag/mobile/new/15November/data/*.jpg'))
for idx in tqdm(range(len(colors))):
    id = int(os.path.basename(colors[idx])[:-4])
    if int(os.path.basename(colors[idx])[:-4]) in [*range(360, 1000, 15)]:
        try:
            cT_a, ret, img = main(colors[idx], camera_matrix_s20, dist_coeff_s20, idx)
            imgs.append(img)
            if not ret: print(id)
            # we need aT_c in formula
            AT_C.append(cT_a.inv())
            MT_F.append(mocap[id])
        except: print('pass on idx: ', colors[idx])

100%|██████████| 2419/2419 [00:13<00:00, 185.85it/s] 


In [95]:
mT_af_q = [0.0, -0.625050,  1.123033, 2.128054, -0.028212, 0.847573, -0.529242, -0.026961]
mT_af = quat_to_SE3(mT_af_q)
AFT_M = mT_af.inv()

In [96]:
AFT_F = []
for mT_f in MT_F:
    AFT_F.append(AFT_M.mul(mrob.geometry.SE3(mT_f)))

In [97]:
AFT_F_ax=[ax(AFT_F[i].T(), 0.05) for i in range(len(AFT_F))]
AT_C_ax=[ax(AT_C[i].T(), 0.1) for i in range(len(AT_C))]
# ax_list.append(ax(mT_af.T(), 0.3))

In [100]:
o3d.visualization.draw_geometries(AFT_F_ax+AT_C_ax)

In [99]:
def load_view_point(coor_frame):

    visualizer = o3d.visualization.Visualizer()
    visualizer.create_window()
    visualizer.add_geometry(coor_frame)
    view_ctl = visualizer.get_view_control()
    view_ctl.set_up((1, 0, 0))  # set the positive direction of the x-axis as the up direction
    view_ctl.set_up((0, -1, 0))  # set the negative direction of the y-axis as the up direction
    view_ctl.set_front((1, 0, 0))  # set the positive direction of the x-axis toward you
    # view_ctl.set_lookat((0, 0, 0))  # set the original point as the center point of the window
    
    
    image = visualizer.capture_screen_float_buffer(True)
    visualizer.destroy_window()

    return image


for index, (each_gt, each_ruco, img) in enumerate(zip(tqdm(MT_F), AT_C, imgs)):
    im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mocap_ax, ruco_ax = ax(each_gt), ax(each_ruco.T())
    m_image = load_view_point(mocap_ax)
    r_image = load_view_point(ruco_ax)

    fig, axs = plt.subplots(1,3,figsize=(16,9))
    axs[0].imshow(m_image)
    axs[0].set_title('Mocap_pose')
    axs[0].set_xticks([])
    axs[0].set_yticks([])
    axs[1].imshow(r_image)
    axs[1].set_title('Ruco_pose')
    axs[1].set_xticks([])
    axs[1].set_yticks([])
    axs[2].imshow(im_rgb)
    axs[2].set_title('Ruco_image')
    axs[2].set_xticks([])
    axs[2].set_yticks([])

    fig.savefig(f"./figures_2/{index:04d}.png")
    plt.close(fig)
    fig.clear()


100%|██████████| 43/43 [01:15<00:00,  1.76s/it]


### in image below, up are poses from mocap and down are poses from ruco-marker. we used opeencv for that. as shown in abowe.

![Getting Started](./Mobile_mocap_poses_before_calibration.png)

$$^{af}T_m \cdot ^{m}T_f = ^{af}T_a \cdot ^{a}T_c \cdot ^{c}T_f$$

$$^{af}T_f = Y \cdot ^{a}T_c \cdot X$$

as defined in the factor:
    $$T_{obs2} =  Y_{origin} \cdot T_{obs1} \cdot X_{target}  $$

In [101]:
def solver(gt, orb):

    gt = np.asarray([i.T() for i in gt])
    orb = np.asarray([i.T() for i in orb])
    Y_ini = mrob.geometry.SE3() #gt[0,:,:]).inv()
    X_ini = mrob.geometry.SE3()

    graph = mrob.FGraph()
    W_0 = 1e0*np.identity(6)
    W = np.identity(6)
    iy = graph.add_node_pose_3d(Y_ini)
    ix = graph.add_node_pose_3d(X_ini)
    #graph.add_factor_1pose_3d(X_ini,ix,W_0)
    #graph.add_factor_1pose_3d(Y_ini,iy,W_0)
    graph.print(True)
    results = graph.get_estimated_state()
    # print('Initial conditions:\n',results[0],'\n',results[1])

    # adding all poses or some of them. Residual is r = Ln ( T_o * T_obs * T_t * T_obs2^-1 )
    # There is aproblem with the second observation... but for gradients
    for i in range(len(gt)):
        T_gt = mrob.geometry.SE3(gt[i,:,:])
        T_orb = mrob.geometry.SE3(orb[i,:,:])
        graph.add_factor_2poses_3d_2obs(obs = T_gt,
                                obs2 = T_orb,
                                nodeOriginId = iy,
                                nodeTargetId = ix,
                                obsInvCov = W)

    graph.solve(mrob.LM, 100, solutionTolerance=1e-6)

    results = graph.get_estimated_state()
    
    # np.save('results.npy', results)

    # afY_a = results[0] 
    # fX_c = results[1]

    # return afY_a, fX_c
    return results


results = solver(AT_C, AFT_F)
print('#'*30)
print('Y= \n', results[0],'\nX=\n',results[1])

Status of graph:  Nodes = 2, Factors = 0, Eigen Factors = 0
Printing NodePose3d: 0, state = 
-0
 0
-0
 0
 0
 0,
 SE3 matrix: 
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1
Printing NodePose3d: 1, state = 
-0
 0
-0
 0
 0
 0,
 SE3 matrix: 
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1
##############################
Y= 
 [[-0.78001109 -0.18958575 -0.59635555  0.29948412]
 [ 0.59867239  0.05129863 -0.79934962 -0.13808568]
 [ 0.18213752 -0.98052317  0.07348626  0.07316656]
 [ 0.          0.          0.          1.        ]] 
X=
 [[-0.34726049  0.32107043  0.88109247 -0.67607491]
 [-0.60545096 -0.79425939  0.05080511  0.17715125]
 [ 0.71612798 -0.51581567  0.4702073   0.50701621]
 [ 0.          0.          0.          1.        ]]

FGraphSolve::optimize_levenberg_marquardt: iteration 1 lambda = 1e-05, error 111.66, and delta = 98.8957
model fidelity = 0.960081 and m_k = 206.015

FGraphSolve::optimize_levenberg_marquardt: iteration 2 lambda = 2.5e-06, error 12.764, and delta = 8.24884
model fidelity = 1.08742 and m_k = 

In [102]:
# help(mrob.FGraph.add_factor_2poses_3d_2obs)

CT_F = mrob.geometry.SE3(results[1])
CT_F.T()

array([[-0.34726049,  0.32107043,  0.88109247, -0.67607491],
       [-0.60545096, -0.79425939,  0.05080511,  0.17715125],
       [ 0.71612798, -0.51581567,  0.4702073 ,  0.50701621],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [105]:
# np.save('FT_C_s20', CT_F.inv().T())

: 

In [103]:
MT_C = []
for mT_fi in MT_F:
    mT_fi = mrob.geometry.SE3(mT_fi)
    mT_c = mT_fi.T() @ CT_F.inv().T()
    MT_C.append(ax(mT_c))

In [104]:
o3d.visualization.draw_geometries(MT_C + AFT_F_ax + AT_C_ax)

so we found calibration. what we do:

take pose from mocap($^{M}T_F$), and  multiply it to inverse of $X$, again $X$ here based on our chain, is $^{C}T_{F}$, frame to camera, and this multiplication will give you pose from Azure camera to mocap, IOW, global pose.

$$^{M}T_C = ^{M}T_F \cdot X^{-1} = ^{M}T_F \cdot ^{C}T_F^{-1}$$



In [22]:
H = np.load('/home/rahim/patata/datasets/navigine/mocab/apriltag/mobile/new/15November/FT_C___mocap_mobile_new.npy')
np.array(H, dtype=np.float16)

array([[-7.021e-01, -5.649e-01,  4.333e-01, -6.128e-02],
       [ 4.807e-01, -8.252e-01, -2.961e-01,  2.756e-01],
       [ 5.249e-01,  3.617e-04,  8.511e-01, -2.771e-03],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  1.000e+00]], dtype=float16)